now that we formulated a thorough model, yet has the notes and everything, we should test it using the formula Root Mean Squared Error (RMSE)

1. take difference between projected and actual stat (error)
2. square difference 
3. mean of the squared difference
4. square root of this mean 

In [1]:
import pandas as pd
import numpy as np

In [2]:
proj_df = pd.read_csv('../nba_csv/player_proj_df.csv')
proj_df.head()

,player_id,proj_ast,proj_blk,proj_dreb,proj_fg3a,proj_fg3m,proj_fga,proj_fgm,proj_fta,proj_ftm,...,ftm,fta,ft_pct,oreb,dreb,ast,tov,stl,blk,pts
0,203932,1.994430,0.710601,5.126943,3.961834,1.402502,13.509175,5.956650,3.608475,2.800268,...,2.4,3.2,0.731,1.7,5.7,3.7,2.1,0.7,0.7,16.0
1,201143,3.247733,1.025916,4.582944,3.079483,1.127251,10.359166,4.860939,1.927194,1.470858,...,1.1,1.4,0.821,1.8,5.0,4.2,1.5,0.9,1.3,13.6
2,202329,1.754433,0.446132,4.286956,3.877699,1.407756,8.190519,3.574506,1.679742,1.238291,...,1.9,2.1,0.867,1.4,6.1,1.3,0.9,0.8,0.4,9.4
3,202692,1.525246,0.143212,1.894640,2.190524,0.692337,7.286938,3.014837,2.166992,1.667793,...,1.8,2.2,0.823,0.5,3.2,2.0,1.0,0.6,0.3,8.8
4,203518,0.988705,0.112903,1.361302,2.800757,1.065089,4.592593,1.876591,0.839953,0.668980,...,0.4,0.4,0.923,0.2,1.4,0.6,0.5,0.5,0.2,5.3


step 1 and 2from above:

In [3]:
proj_df['squared_error'] = (proj_df['proj_pts'] - proj_df['pts']) ** (2)

step 3:

In [4]:
mse = proj_df['squared_error'].mean()

step 4: 

In [5]:
rmse = mse ** (1/2) #square rooting

In [6]:
rmse

3.302971088916984

doing same for assists to test

In [7]:
proj_df['squared_error'] = (proj_df['proj_ast'] - proj_df['ast'])**(2)
mse = proj_df['squared_error'].mean()
rmse = mse ** (1/2)
rmse

0.9944342105776937

our ast number (0.994ish) was closer than points rmse number, smaller = proj closer

now will utilize sklearn and it's mse func

In [8]:
from sklearn.metrics import mean_squared_error #getting func from lib
from math import sqrt

In [9]:
#takes two obj: lists for predicted and actual values
player_proj = pd.read_csv('../nba_csv/player_proj_df.csv')
player_info = pd.read_csv('../nba_csv/player_info_df.csv')

splitting proj df into two df's: real proj (true values), and our proj (predicted test values) to use in rmse method

In [10]:
df_real = player_proj.loc[:, ['pts','min','fgm','fga','fg3m','fg3a','ftm','fta','oreb','dreb','ast','stl','tov', 'blk']]
df_proj = player_proj.loc[:, ['proj_pts','proj_min','proj_fgm','proj_fga','proj_fg3m','proj_fg3a','proj_ftm','proj_fta','proj_oreb','proj_dreb','proj_ast','proj_stl','proj_tov', 'proj_blk']]

#splitting the df's, we want all the rows, but ofc splitting cols into real and proj

In [11]:
#now will be doing rmse
se = mean_squared_error(df_real, df_proj)
mse = np.mean(se)
rmse = np.sqrt(mse)
confidence = rmse

In [12]:
#print statement showing confidence in model
#seen by: confidence = 1 - error basically

print('{0} percent confidence in projected {1} per game stats'.format(100 - round(confidence, 2), '2018-19'))

98.03 percent confidence in projected 2018-19 per game stats


concatinating the info and name df's to see the names along with what we just did

In [13]:
df_real_stats = pd.concat([player_info, df_real], axis = 1)
df_proj_stats = pd.concat([player_info, df_proj], axis = 1)

renaming real stats col of proj_season_id to season_id

In [14]:
df_real_stats.rename(
    columns = 
    {'proj_season_id' : 'season_id'}, inplace = True)

making it easier for comparison, renaming all proj columns (removing 'proj_' prefix since the df named proj_stats)

In [15]:
df_proj_stats.rename(columns = {'proj_season_id':'season_id',
                               'proj_pts': 'pts',
                                'proj_min':'min',
                                'proj_fgm':'fgm',
                                'proj_fga':'fga',
                                'proj_fg3m':'fg3m',
                                'proj_fg3a':'fg3a',
                                'proj_ftm':'ftm',
                                'proj_fta':'fta',
                                'proj_oreb':'oreb',
                                'proj_dreb':'dreb',
                                'proj_ast':'ast',
                                'proj_stl':'stl',
                                'proj_tov':'tov',
                               'proj_blk':'blk'}, inplace = True)

## comparing it to competitor 

from fantasy pros, comparison using the derived rmse 

In [16]:
player_proj = pd.read_csv('../nba_csv/player_proj_df.csv')

In [17]:
#since our model and FantasyPro model don't have same columns, let's pull the ones we need
columns = ['player_name',
           'player_id',
           'proj_pts',
           'proj_min',
           'proj_fgm',
           'proj_fga',
           'proj_fg3m',
           'proj_fg3a',
           'proj_ftm',
           'proj_fta',
           'proj_oreb',
           'proj_dreb',
           'proj_ast',
           'proj_stl',
           'proj_tov',
           'proj_blk']

#new df that has the projections, the final
df_proj_final = player_proj.loc[:, columns]

In [18]:
#need fg and ft%

df_proj_final['proj_fg%'] = df_proj_final['proj_fgm'] / df_proj_final['proj_fga']
df_proj_final['proj_ft%'] = df_proj_final['proj_ftm'] / df_proj_final['proj_fta']

#along with overall proj rebounds (o+dreb)

df_proj_final['proj_reb'] = df_proj_final['proj_oreb'] + df_proj_final['proj_dreb']

In [19]:
#now, we update and create our REAL final df

final_columns = [
    'player_name',
           'player_id',
           'proj_pts',
           'proj_reb',
           'proj_ast',
           'proj_blk',
           'proj_stl',
           'proj_fg%',
           'proj_ft%',
           'proj_fg3m',
           'proj_min',
           'proj_tov'
]

df_proj_final = df_proj_final[final_columns]

In [20]:
df_competitor = pd.read_csv('../nba_csv/ESPN_CBS_FantasyPros_Fantasy_Basketball_Overall_2018_Average_Projections.csv')

In [21]:
#cleaning up unnecessary cols for comparison

columns_to_drop = [
    'Team',
    'Positions',
    'GP'
]

df_competitor.drop(columns = columns_to_drop, inplace = True)#overriding df 


only have player name, but we need player id. Let's lowercase all names and merge df's


In [22]:
lowercase_names = df_competitor['Player'].str.lower()
df_competitor['Player'] = lowercase_names


player_info = pd.read_csv('../nba_csv/player_info_df.csv')
season = player_info['proj_season_id'] == '2018-19'
player_df = player_info[season]
lowercase = player_df['player_name'].str.lower()
player_df['player_name'] = lowercase


In [23]:
comp_merged = pd.merge(df_competitor,
        player_df[['player_name', 'player_id']],
        how='left',
        left_on = 'Player',
        right_on = 'player_name').drop_duplicates().reset_index(drop=True)

we can rename player for the df_competitor to player_name, but left_on and right_on little easier

In [24]:
#dropping rows with missings vals (just in case :) )

comp_merged.dropna(how = 'any', inplace = True)

In [25]:
comp_merged.sample(5)

,Player,PTS,REB,AST,BLK,STL,FG%,FT%,3PM,MIN,TO,player_name,player_id
315,vince carter,4.8,2.0,1.6,0.3,0.5,0.396,0.788,0.8,16.6,0.6,vince carter,1713.0
214,terry rozier,7.6,3.6,2.1,0.1,0.7,0.396,0.778,1.3,24.9,0.7,terry rozier,1626179.0
179,kenneth faried,8.8,5.6,0.9,0.7,0.7,0.531,0.689,0.0,20.7,1.1,kenneth faried,202702.0
185,fred vanvleet,8.7,2.3,3.0,0.3,0.8,0.421,0.837,1.4,21.5,1.0,fred vanvleet,1627832.0
15,bradley beal,23.0,3.8,4.1,0.4,1.1,0.474,0.805,2.7,35.9,2.3,bradley beal,203078.0


In [26]:
#running a sample, we see that player_id is float for some reason
#changing to int 

player_ids = comp_merged['player_id'].astype(int)
comp_merged['player_id'] = player_ids

#also, we have 'Player' column still, but we don't need

del comp_merged['Player']

In [27]:
#getting order how we want it
cols = [
    'player_name',
    'player_id',
    'PTS',
    'REB',
    'AST',
    'BLK',
    'STL',
    'FG%',
    'FT%',
    '3PM',
    'MIN',
    'TO']

comp_merged = comp_merged[cols]

In [28]:
df_real = player_proj.loc[:,['player_name', 'player_id', 'pts','min','fgm','fga','fg3m','fg3a','ftm','fta','oreb','dreb','ast','stl','tov', 'blk']]

In [29]:
df_real['fg%'] = df_real['fgm']  / df_real['fga']
df_real['ft%'] = df_real['ftm'] / df_real['fta']
df_real['reb'] = df_real['oreb'] + df_real['dreb']

#creating this cols since they don't exist here 

In [30]:
final_real_columns = [
    'player_name',
           'player_id',
           'pts',
           'reb',
           'ast',
           'blk',
           'stl',
           'fg%',
           'ft%',
           'fg3m',
           'min',
           'tov'
]

#dropping unnecessary columns, filtering df

In [31]:
df_real_final = df_real[final_real_columns]

In [32]:
df_real_final = round(df_real_final, 2)
df_real_final.head()

,player_name,player_id,pts,reb,ast,blk,stl,fg%,ft%,fg3m,min,tov
0,Aaron Gordon,203932,16.0,7.4,3.7,0.7,0.7,0.45,0.75,1.6,33.8,2.1
1,Al Horford,201143,13.6,6.8,4.2,1.3,0.9,0.54,0.79,1.1,29.0,1.5
2,Al-Farouq Aminu,202329,9.4,7.5,1.3,0.4,0.8,0.44,0.90,1.2,28.3,0.9
3,Alec Burks,202692,8.8,3.7,2.0,0.3,0.6,0.41,0.82,1.0,21.5,1.0
4,Alex Abrines,203518,5.3,1.6,0.6,0.2,0.5,0.35,1.00,1.3,19.0,0.5


merging competitor df with our df by player_id, to use rmse func

In [33]:
competitor_final = pd.merge(comp_merged, df_real_final, how = 'left', on = 'player_id')
competitor_final.sample(5)

,player_name_x,player_id,PTS,REB,AST,BLK,STL,FG%,FT%,3PM,...,pts,reb,ast,blk,stl,fg%,ft%,fg3m,min,tov
93,kentavious caldwell-pope,203484,12.2,3.7,2.0,0.2,1.2,0.433,0.808,1.9,...,11.4,2.9,1.3,0.2,0.9,0.43,0.89,1.8,24.8,0.8
256,miles plumlee,203101,3.4,3.1,0.6,0.3,0.3,0.532,0.525,0.0,...,4.4,2.2,0.9,0.2,0.3,0.67,0.53,0.0,9.6,0.6
169,marquese chriss,1627737,7.7,4.5,0.8,0.8,0.8,0.433,0.615,1.0,...,4.2,3.3,0.5,0.3,0.4,0.38,0.70,0.4,11.6,0.8
137,michael kidd-gilchrist,203077,9.2,5.6,1.2,0.7,0.9,0.482,0.734,0.3,...,6.7,3.9,1.0,0.6,0.5,0.48,0.79,0.3,18.4,0.7
108,wesley matthews,202083,11.3,2.8,2.4,0.2,1.0,0.408,0.819,1.4,...,12.2,2.5,2.3,0.2,0.8,0.40,0.79,2.2,30.3,1.3


In [34]:
#dropping columns w null vals

competitor_proj = competitor_final.dropna(how = 'any')

slicing into df_real and projection

In [35]:
df_real = competitor_proj.loc[:, ['pts','reb','ast','blk','stl','fg%','ft%','fg3m','min','tov']]
df_proj = competitor_proj.loc[:, ['PTS','REB','AST','BLK','STL','FG%','FT%','3PM','MIN','TO']]
#caps is projected cols, lower is real cols

In [36]:
#running into mse func
from sklearn.metrics import mean_squared_error
lin_mse = mean_squared_error(df_real, df_proj)
lin_rmse = np.mean(lin_mse)
confidence = np.sqrt(lin_rmse)
print('{0} percent confidence in projected {1} per game stats'.format(100 - round(confidence, 2), '2018-19'))

#for fantasy pro guys

97.4 percent confidence in projected 2018-19 per game stats


seeing how we compare

In [37]:
df_proj_final.sample(5)
df_real_final.sample(5)


,player_name,player_id,pts,reb,ast,blk,stl,fg%,ft%,fg3m,min,tov
184,Larry Nance Jr.,1626204,9.4,8.2,3.2,0.6,1.5,0.52,0.70,0.5,26.8,1.4
284,Trevor Ariza,2772,12.5,5.4,3.7,0.3,1.3,0.40,0.79,2.1,34.0,1.5
233,Pau Gasol,2200,3.9,4.5,1.7,0.5,0.2,0.45,0.69,0.2,12.0,0.5
135,Jeff Green,201145,12.3,4.0,1.8,0.5,0.6,0.47,0.89,1.4,27.2,1.3
279,Tobias Harris,202699,20.0,7.8,2.8,0.5,0.6,0.49,0.86,1.9,34.7,1.8


In [38]:
model_final = pd.merge(df_real_final, df_proj_final, how = 'left', on = 'player_id')
model_final.head()

,player_name_x,player_id,pts,reb,ast,blk,stl,fg%,ft%,fg3m,...,proj_pts,proj_reb,proj_ast,proj_blk,proj_stl,proj_fg%,proj_ft%,proj_fg3m,proj_min,proj_tov
0,Aaron Gordon,203932,16.0,7.4,3.7,0.7,0.7,0.45,0.75,1.6,...,16.063873,6.748596,1.994430,0.710601,0.909579,0.440934,0.776026,1.402502,33.942109,1.579757
1,Al Horford,201143,13.6,6.8,4.2,1.3,0.9,0.54,0.79,1.1,...,12.340232,5.801251,3.247733,1.025916,0.652499,0.469240,0.763213,1.127251,30.246673,1.609267
2,Al-Farouq Aminu,202329,9.4,7.5,1.3,0.4,0.8,0.44,0.90,1.2,...,9.784116,5.388717,1.754433,0.446132,0.946406,0.436420,0.737191,1.407756,27.978945,1.138945
3,Alec Burks,202692,8.8,3.7,2.0,0.3,0.6,0.41,0.82,1.0,...,8.349126,2.417396,1.525246,0.143212,0.541122,0.413732,0.769635,0.692337,20.803754,1.014058
4,Alex Abrines,203518,5.3,1.6,0.6,0.2,0.5,0.35,1.00,1.3,...,5.502404,1.731801,0.988705,0.112903,0.481071,0.408612,0.796450,1.065089,16.998044,0.568089


In [39]:
df_real = model_final.loc[:, ['pts','reb','ast','blk','stl','fg%','ft%','fg3m','min','tov']]
df_proj = model_final.loc[:, ['proj_pts','proj_reb','proj_ast','proj_blk','proj_stl','proj_fg%','proj_ft%','proj_fg3m','proj_min','proj_tov']]

#filling vals
df_real.fillna(0, inplace=True)
df_proj.fillna(0, inplace=True)

#filling by 0 to deal with division by 0 error

# calculate mean square error
lin_mse = mean_squared_error(df_real, df_proj)
lin_rmse = np.mean(lin_mse)
confidence = np.sqrt(lin_rmse)
print('{0} percent confidence in projected {1} per game stats'.format(100 - round(confidence, 2), '2018-19'))

97.88 percent confidence in projected 2018-19 per game stats


## Modifying Variables

note to future me, 

essentially, we can change weights in player comparison func like below: 

def player_comparison_tool(df, current_player_season, current_player_id):
    if (((df['season_id'] == current_player_season) & (df['player_id'] == current_player_id)).any() == False):
        print('Can\'t find player with id {0} and season {1}'.format(current_player_id, current_player_season))
        return
    # might need to return None if so
    for row in df.itertuples():
        if current_player_season == row.season_id and current_player_id == row.player_id:
            current_player_id = row.player_id
            break

    current_player_vector = np.array([
        (df.loc[(df['player_id'] == current_player_id) & (df['season_id'] == current_player_season), 'pts_norm']).item(),
        (df.loc[(df['player_id'] == current_player_id) & (df['season_id'] == current_player_season), 'min_norm']).item(),
        (df.loc[(df['player_id'] == current_player_id) & (df['season_id'] == current_player_season), 'fgm_norm']).item(),
        (df.loc[(df['player_id'] == current_player_id) & (df['season_id'] == current_player_season), 'fga_norm']).item(),
        (df.loc[(df['player_id'] == current_player_id) & (df['season_id'] == current_player_season), 'fg3m_norm']).item(),
        (df.loc[(df['player_id'] == current_player_id) & (df['season_id'] == current_player_season), 'fg3a_norm']).item(),
        (df.loc[(df['player_id'] == current_player_id) & (df['season_id'] == current_player_season), 'ftm_norm']).item(),
        (df.loc[(df['player_id'] == current_player_id) & (df['season_id'] == current_player_season), 'fta_norm']).item(),
        (df.loc[(df['player_id'] == current_player_id) & (df['season_id'] == current_player_season), 'oreb_norm']).item(),
        (df.loc[(df['player_id'] == current_player_id) & (df['season_id'] == current_player_season), 'dreb_norm']).item(),
        (df.loc[(df['player_id'] == current_player_id) & (df['season_id'] == current_player_season), 'ast_norm']).item(),
        (df.loc[(df['player_id'] == current_player_id) & (df['season_id'] == current_player_season), 'stl_norm']).item(),
        (df.loc[(df['player_id'] == current_player_id) & (df['season_id'] == current_player_season), 'tov_norm']).item(),
        (df.loc[(df['player_id'] == current_player_id) & (df['season_id'] == current_player_season), 'blk_norm']).item()
    ])

    print('Projecting player_id {0} for season {1}'.format(current_player_id, season_list[(season_list.index(row.season_id) + 1)]))
    
    # create a list to store the data
    player_distance = []

    # loop over every row in the dataframe to calculate percent error
    weighted_numbers = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
    for row in df.itertuples():
        compared_player_vector = np.array([
        row.pts_norm,
        row.min_norm,
        row.fgm_norm,
        row.fga_norm,
        row.fg3m_norm,
        row.fg3a_norm,
        row.ftm_norm,
        row.fta_norm,
        row.oreb_norm,
        row.dreb_norm,
        row.ast_norm,
        row.stl_norm,
        row.tov_norm,
        row.blk_norm
        ])
        
        vfunc = np.vectorize(calc_distance)
        distance_vect = vfunc(current_player_vector, compared_player_vector)
        weighted_distance = distance_vect * weighted_numbers
        number = np.sum(weighted_distance)
        player_distance.append(number)
        
    # create a new column with error 
    df['distance'] = player_distance

    # sort dataframe by smallest distance
    ranked_df = df.sort_values('distance')
    
    stats = ['pts',
             'min',
             'fgm',
             'fga',
             'fg3m',
             'fg3a',
             'ftm',
             'fta',
             'oreb',
             'dreb',
             'ast',
             'stl',
             'tov',
             'blk'
             ]
    
    # create empty dictionary to put in projected stats
    projected_stats = {}

    for col in stats:
        sum_stat = 0
        sum_weight = 0
        for index, row in ranked_df.iloc[1:11].iterrows():
            # skip over the row if it was 2017-18 season because we can't take the next
            if row.season_id == '2017-18':
                continue
            # get the players next season
            weight = (1 / row.distance)
            next_season = season_list[(season_list.index(row.season_id) + 1)]
            # find the player row with the id and the next season
            player_next_season = find_player(ranked_df, row.player_id, next_season)
            # if player_next_season doesn't exist then skip
            if player_next_season == None:
                continue
            sum_stat += getattr(player_next_season, col) * weight
            sum_weight += weight
        projected_stats['player_id'] = current_player_id
        projected_stats['proj_season_id'] = season_list[(season_list.index(current_player_season) + 1)]
        projected_stats['proj_' + col] = (sum_stat / sum_weight)
    return projected_stats

to change values when running rmse func, to get closer in confidence

of course, we can also change base df, i.e. 36 min or per 100 poss  